## 属性连接3-计算布里渊指数

In [69]:
import pandas as pd
import numpy as np
import math

### 读取压缩后的数据

In [40]:
%%time
select = pd.read_csv('./result/words_selected_result.csv')
select

CPU times: user 10.6 s, sys: 541 ms, total: 11.2 s
Wall time: 11.2 s


,0,1,2,3,4,5,6,7,8,9,...,15107,15108,15109,15110,15111,15112,15113,15114,15115,15116
0,absorb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,advantag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,biolog,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,bodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,capabl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409,uid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2410,blockchain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2411,ofdm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2412,lstm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 读取属性表

In [41]:
%%time
props = pd.read_excel('./dataset/prop3.xlsx')
props

CPU times: user 962 ms, sys: 6.02 ms, total: 968 ms
Wall time: 972 ms


,ID,PN,技术领域
0,1,US2011063416,"8,9,2"
1,2,US2010117799,"3,6"
2,3,JP2011002997,"6,7"
3,4,JP2010246671,"13,6"
4,5,WO2010082246,"13,6,7"
...,...,...,...
15111,15112,EP3878357,13
15112,15113,US2021267325,"1,34,6"
15113,15114,TW616087,"6,7"
15114,15115,CN112773347,13


### 制作字典

In [42]:
def create_dict(props):
    prop_dict = {}
    for i in range(len(props)):
        id = props.iloc[i, 0]
        field = props.iloc[i, 2]
        if not pd.isnull(field):
            fields = str(field).strip(' ').split(',')
            prop_dict[id] = fields
    return prop_dict

In [43]:
%%time
prop_dict = create_dict(props)
prop_dict

CPU times: user 655 ms, sys: 5.02 ms, total: 660 ms
Wall time: 660 ms


{1: ['8', '9', '2'],
 2: ['3', '6'],
 3: ['6', '7'],
 4: ['13', '6'],
 5: ['13', '6', '7'],
 6: ['13', '15'],
 7: ['13', '7'],
 8: ['4', '13'],
 9: ['2', '6', '7', '11', '13'],
 10: ['7'],
 11: ['13', '6'],
 12: ['6'],
 13: ['2', '6'],
 14: ['1', '2', '6', '8', '17', '19', '21', '22'],
 15: ['4', '6'],
 16: ['7'],
 17: ['13'],
 18: ['11', '6'],
 19: ['2', '4', '6'],
 20: ['13', '6', '7'],
 21: ['12', '13'],
 22: ['6', '7'],
 23: ['6', '7'],
 24: ['6', '15'],
 25: ['6', '7'],
 26: ['11', '13', '15'],
 27: ['16', '11', '15'],
 28: ['16', '11', '15'],
 29: ['11', '6', '7'],
 30: ['16', '11', '14', '15'],
 31: ['10', '11', '13', '15', '22'],
 32: ['6', '14', '15'],
 33: ['13'],
 34: ['3', '13'],
 35: ['13', '7'],
 36: ['6'],
 37: ['4', '13', '6', '7'],
 38: ['13'],
 39: ['6', '7'],
 40: ['6', '7'],
 41: ['13'],
 42: ['2'],
 43: ['13', '6', '7'],
 44: ['2', '4', '6', '7'],
 45: ['3'],
 46: ['13', '6', '7'],
 47: ['13', '7'],
 48: ['13', '6'],
 49: ['13', '7'],
 50: ['7'],
 51: ['13'],
 52: 

### 映射处理技术领域

In [87]:
def mapping_fields(select, prop_dict):
    res = []
    for i in range(len(select)):
        fields = {}
        row = select.iloc[i, :].values
        # row[0]为单词 其他为id号
        N = 0
        for j in range(1, len(row)):
            if (row[j] > 0.0): # 如果列大于0就去取技术领域
                values = prop_dict.get(j)
                if values is not None:
                    N += len(values)
                    for k in values:
                        if fields.__contains__(k):
                            fields[k] = fields.get(k)+1
                        else: fields[k] = 1
        # 计算布里渊指数-初始化为空
        brillouin = None
        if N != 0: # 总数量不为0再计算！
            sum = 0
            for key in fields:
                time = fields[key]
                sum += math.log10(np.math.factorial(time))
            brillouin = (math.log10(np.math.factorial(N)) - sum) / N
        res.append([row[0], fields, N, brillouin]) # 保存单词-技术领域分类统计-所有数量（不去重）-布里渊
        
    return pd.DataFrame(res, columns=['word', 'fields', 'sum', 'brillouin'])

In [84]:
%%time
res = mapping_fields(select, prop_dict)
res

CPU times: user 19.4 s, sys: 36.7 ms, total: 19.5 s
Wall time: 19.5 s


,word,fields,sum,brillouin
0,absorb,"{'13': 6, '16': 1, '23': 1, '8': 1, '2': 3, '1...",16,0.586507
1,advantag,{},0,NaN
2,biolog,"{'3': 7, '6': 21, '7': 4, '10': 2, '11': 2, '1...",86,0.653747
3,bodi,"{'13': 65, '7': 9, '3': 11, '4': 21, '6': 33, ...",179,0.832799
4,capabl,"{'3': 3, '13': 2, '6': 2, '4': 7, '7': 5, '11'...",20,0.561215
...,...,...,...,...
2409,uid,"{'33': 1, '13': 6, '25': 1, '9': 2, '10': 1, '...",14,0.500278
2410,blockchain,"{'4': 37, '13': 68, '6': 64, '7': 25, '3': 1, ...",198,0.586711
2411,ofdm,"{'4': 6, '3': 2}",8,0.180895
2412,lstm,"{'13': 5, '6': 4}",9,0.233375


### 生成结果文件

In [85]:
# 传入df和文件名生成csv文件
def save_file(df, file_name):
    df.to_csv('./result/' + file_name + 'result.csv', index=False)

In [86]:
%%time
save_file(res, 'prop_connect3_')

CPU times: user 18.3 ms, sys: 3.09 ms, total: 21.4 ms
Wall time: 22.4 ms
